In [1]:
import pandas as pd
import numpy as np

In [2]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def get_phrog(row):
    """ get phrog cluster """
    if 'phrog_' in str(row['target']):
        return int(row['target'].split('_')[-1])
    else:
        return 0
    
    
def curate_columns(row):
    
    phrog = str(row['phrog'])
    alan_profile = str(row['alan_profile'])
    
    if phrog != '0': return phrog
    elif alan_profile != '0': return alan_profile
    else: return '0'

In [3]:
print('Load tables for testing... ', end='')


phrogs_df = pd.read_csv('tables/phrogs.tsv', sep='\t')
alan_df = pd.read_csv('tables/alan.tsv', sep='\t')
pfam_df = pd.read_csv('tables/pfam.tsv', sep='\t')
ecod_df = pd.read_csv('tables/ecod.tsv', sep='\t')

### metadata tables


### replace PHROGS functions to CUSTOM MGG PHROGS functions.

# load
phrogs_annot_df = pd.read_csv('tables/phrog_annot_v4.tsv', sep='\t') # load PHROGS
mgg_phrogs_annot_df = pd.read_csv('tables/v3_phrogs-table-rafal-3_12.csv', sep=';') # load MGG PHROGS

# rename columns
mgg_phrogs_annot_df = mgg_phrogs_annot_df.rename(columns={'funct.orig': 'annot', 'funct.new': 'annot_mgg'})[['annot', 'annot_mgg']]

# map to CUSTOM MGG PHROGS functions
phrogs_annot_df.merge(mgg_phrogs_annot_df, on='annot', how='left') \
               .drop('annot', axis=1) \
               .rename(columns={'annot_mgg': 'annot'})

# format
phrogs_annot_df = phrogs_annot_df[['phrog', 'color', 'annot', 'category']] # order columns
phrogs_annot_df = phrogs_annot_df.fillna('unknown function')


### ALANDB
alan_annot_df = pd.read_csv('tables/alan_annot.tsv', sep=',')
alan_annot_df = alan_annot_df.rename(columns={'definition': 'annot', 'funct': 'category', 'profile': 'alan_profile'})
alan_annot_df = alan_annot_df.drop(['abbrev', 'hmm.name', 'family.name', 'family.name.base', 'where.it.occurs', 'is.custom', 'custom.hmm.exists'], axis=1)

print('Done!')


print('Report hits... ', end='')
print('PHROGS metadata... ', end='')
phrogs_df['phrog'] = phrogs_df.apply(get_phrog, axis=1)
phrogs_df = phrogs_df.merge(phrogs_annot_df, on='phrog', how='left')

print('ALANDB metadata... ', end='')
alan_df['alan_profile'] = alan_df['target']
alan_df = alan_df.merge(alan_annot_df, on='alan_profile', how='left')

print('final table... ', end='')
df = pd.concat([phrogs_df, alan_df, pfam_df, ecod_df], axis=0)
df[['phrog', 'alan_profile']] = df[['phrog', 'alan_profile']].fillna('0')
df['phrog'] = df['phrog'].astype(int)
df['phrog/alan_profile'] = df.apply(curate_columns, axis=1)
df = df.drop(['phrog', 'alan_profile'], axis=1)


print('Done!')

Load tables for testing... Done!
Report hits... PHROGS metadata... ALANDB metadata... final table... Done!


In [4]:
df.head(3)

,query,target,prob,pvalue,ident,qcov,tcov,bits,qstart,qend,...,tstart,tend,tlength,evalue,db,name,color,annot,category,phrog/alan_profile
0,PC00001,phrog_384,1.000,2.900000e-45,0.19,0.94,0.96,250.0,11,154,...,5,154,157,2.200000e-41,PHROGS,phrog_384 ## p257441 VI_10110,#3e83f6,virion structural protein,head and packaging,384
1,PC00001,phrog_11600,0.996,1.500000e-23,0.19,0.99,0.32,146.8,1,153,...,1,150,472,1.300000e-19,PHROGS,phrog_11600 ## MG099933_p26,#c9c9c9,unknown function,unknown function,11600
2,PC00001,phrog_9812,0.571,1.100000e-04,0.40,0.10,0.18,26.0,57,71,...,2,16,83,9.400000e-01,PHROGS,phrog_9812 ## NC_009878_p42,#c9c9c9,unknown function,unknown function,9812


In [176]:
def get_confidence_column(df, eval_intervals=(10**-10, 10**-5, 10**-3, 1), col='evalue', verbose=False):
    
    conditions, choices = [], []

    # df = pd.DataFrame({'evalue': [10**-10, 10**-7, 10**-3, 10, 10**2]})
    
    # get conditions
    for i, evalue in enumerate(eval_intervals):
        # lower than first number
        if i == 0:
            conditions.append((df[col] <= evalue))
            if verbose: print(f'<= {evalue:.2E}')

        # between numbers
        else:
            lower_evalue = eval_intervals[i-1]
            conditions.append((df[col] > lower_evalue) & (df[col] <= evalue))
            if verbose: print(f'> {lower_evalue:.2E} and <= {evalue:.2E}')
    
    # higher than last number
    conditions.append(df[col] >= eval_intervals[-1])
    if verbose: print(f'> {eval_intervals[-1]}')
    

    choices = ['*' * i for i in range(len(eval_intervals)).__reversed__()]
    choices = choices[:-1] + ['!', '!']

    if verbose: print(choices)
    if verbose: np.select(conditions, choices, default='?')

    return np.select(conditions, choices, default='?')


def get_no_hit_row(n, columns_mapper = {'query': 'string', 'target': 'string', 'prob': 'float', \
                                        'pvalue': 'float', 'ident': 'float', 'qcov': 'float', \
                                        'tcov': 'float', 'bits': 'float', 'qstart': 'int', \
                                        'qend': 'int', 'qlength': 'int', 'tstart': 'int', \
                                        'tend': 'int', 'tlength': 'int', 'evalue': 'float', \
                                        'db': 'string', 'name': 'string', 'color': 'string', \
                                        'annot': 'string', 'category': 'string', 'phrog/alan_profile': 'string',
                                        'report_label': 'string', 'report_function': 'string', 'report_params': 'string'}):
    
    """ Give dict of column names and variable types to create 'no hit' row as data frame object """

    values, indicies = [], columns_mapper.keys()
    for key, variable_type in columns_mapper.items():
        if variable_type == 'string': values.append('-')
        else: values.append(0)

    no_hit_row = pd.Series(values, index=indicies).to_frame().T
    no_hit_row = pd.concat([no_hit_row]*n)
    return no_hit_row


def report_pfam(df, eval_intervals=(10**-10, 10**-5, 10**-3, 1)):
    
    # remove domains of unknown functions
    filt_not_duf = ~ (df['name'].str.contains("DUF"))
    df = df.loc[filt_not_duf].copy()
    
    # get confidence
    confidence = get_confidence(eval_intervals)

    # report best bit score for lowest evalues (stepwise)
    for confidence, significance in zip(confidence_intervals, eval_intervals):
        try:
            name, evalue, bits = df.loc[df.query('evalue <= @significance')['bits'].idxmax(), ['name', 'evalue', 'bits']].to_list()
            pfamID, func_short, func_detailed = name.split(';')[0].strip(), name.split(';')[1].strip(), name.split(';')[2].strip()
            report = f'{confidence} {func_detailed} [{func_short}] ({pfamID}) ({evalue:.2E}, {int(bits)})'
            break
        
        except ValueError:
            report = 'no significant hit'

    return report


def report_ecod(df, eval_intervals=(10**-10, 10**-5, 10**-3, 1)):
    """ from the most sigfinicat to the none-significat """

    # report best bit score for lowest evalues (stepwise)
    for confidence, significance in zip(confidence_intervals, eval_intervals):
        try:
            name, evalue, bits = df.loc[df.query('evalue <= @significance')['bits'].idxmax(), ['name', 'evalue', 'bits']].to_list()
            
            F_INDEX, ecod_levels = name.split('|')[1].strip(), name.split('|')[3]
            T, F = ecod_levels.split(': ')[4].strip(', F'), ecod_levels.split(': ')[5].strip()

            report = f'{confidence} T: {T}, F: {F} ({F_INDEX}) ({evalue:.2E}, {int(bits)})'
            break
        
        except ValueError:
            report = 'no significant hit'

    return report


def report_phrogs(df, max_evalue=10**-3, nfunc2report=2, verbose=False):
    
    """
    1. Filter eval 10**-3
    2. Remove unknown function [REPORT ONLY WHEN NO OTHER FUNCTION, PRIORITY-0]
    3. Remove non-informative functions (lytic tail protein, tail protein, structural protein, virion structural protein, minor tail protein ...) [REPORT ONLY WHEN NO OTHER FUNCTION; PRIORITY-1]
    4. Group by unique functions. For each function report independently max bitscore and max qcov (hits to this function).
    5. Take two functions with highest bitscores.
    6. Report {confidence} {function} in one genbank field, and in seperate field bitscore and qcov.
    7. Report two best PHROG hits seperataly (in total four PHROGS field: 2x function with confidence and 2x params: bitscore and qcov) [PRIORITY-2]
    """
    # get PC name
    pcid = df['query'].unique()[0]
    
    #### get informative hits
    noninformative_functions = ['lytic tail protein', 'tail protein', 'structural protein', 'virion structural protein', 'minor tail protein']

    filt_evalue = 'evalue <= @max_evalue'
    filt_unknown = 'annot != "unknown function"'
    fitl_noninformative_functions = '~(annot.isin(@noninformative_functions))'

    query = ' and '.join([filt_evalue, filt_unknown, fitl_noninformative_functions])
    df = df.query(query)

    # select best hits for each unique function (highest bitscore)
    best_hits_df = df.loc[df.groupby('annot')['bits'].idxmax()] \
                     .sort_values('bits', ascending=False) \
                     .copy()
    
    # get highest qcov for each unique function
    best_qcov_df = df.loc[df.groupby('annot')['qcov'] \
                            .idxmax()][['annot','qcov']] \
                            .copy()
    
    # best hits for each unique function & highest qcov for given function
    final_df = best_hits_df.merge(best_qcov_df, on='annot', how='left', suffixes=('_oryginal', '_best')) \
                           .drop('qcov_oryginal', axis=1) \
                           .rename(columns={'qcov_best': 'qcov'}) \
                           .sort_values('bits', ascending=False)
    
    
    ### report function    
    top_hits_df = final_df.iloc[:nfunc2report].copy()
    
    # prepare columns2report
    if len(top_hits_df) == 0: 
        top_hits_df = get_no_hit_row(n=2)    
    elif len(top_hits_df) == 1: 
        holder_row = get_no_hit_row(n=1)
        top_hits_df = pd.concat([top_hits_df, holder_row])
    else: pass

    # report columns
    labels = [f'PHROGS{i}' for i in range(1,len(top_hits_df)+1)]
    
    top_hits_df['query'] = [pcid] * len(top_hits_df)
    top_hits_df['report_label'] = labels
    top_hits_df['report_function'] = top_hits_df['annot']
    top_hits_df['report_params'] = top_hits_df.apply(lambda row: f'bits: {int(row["bits"]): <4} qcov: {int(row["qcov"]):.2f}', axis=1)
    top_hits_df['report_confidence'] = get_confidence_column(top_hits_df)
            
    if verbose: display(top_hits_df)
    
    return top_hits_df



def report_alan(df, min_prob=0.95, nfunc2report=2, verbose=True):
    """ ... """
    
    # get PC name
    pcid = df['query'].unique()[0]
    
    # sort significant hits
    filter_df = df.query('prob >= @min_prob').sort_values('bits', ascending=False)
    
    ### report function    
    top_hits_df = filter_df.iloc[:nfunc2report].copy()
    
       # prepare columns2report
    if len(top_hits_df) == 0: 
        top_hits_df = get_no_hit_row(n=2)    
    elif len(top_hits_df) == 1: 
        holder_row = get_no_hit_row(n=1)
        top_hits_df = pd.concat([top_hits_df, holder_row])
    else: pass

    # report columns
    labels = [f'ALAN{i}' for i in range(1,len(top_hits_df)+1)]
    
    top_hits_df['query'] = [pcid] * len(top_hits_df)
    top_hits_df['report_label'] = labels
    top_hits_df['report_function'] = top_hits_df['category']
    top_hits_df['report_params'] = top_hits_df.apply(lambda row: f'bits: {int(row["bits"]): <4} eval: {row["evalue"]:.2E}', axis=1)
    top_hits_df['report_confidence'] = get_confidence_column(top_hits_df)
            
    if verbose: display(top_hits_df)

    return top_hits_df

In [177]:
best_hits_dfs = []
for pcid, pc in df.groupby('query'):    
    for dbid, db in pc.groupby('db'):
        
        # report function
        if dbid == 'PHROGS': phrogs_df = report_phrogs(db, max_evalue=10**-3, nfunc2report=2, verbose=False)
        elif dbid == 'ALANDB': alan_df = report_alan(db, min_prob=0.95, nfunc2report=2, verbose=False)
        # elif dbid == 'PFAM': pfam = report_pfam(db)
        # elif dbid == 'ECOD': ecod = report_ecod(db)
        else: pass

    best_hits_dfs.append(alan_df)
    best_hits_dfs.append(phrogs_df)

    # if pcid == 'PC00005': break

In [178]:
best_hits_df = pd.concat(best_hits_dfs).reset_index(drop=True)
best_hits_df['report_params'].to_list()

['bits: 182  eval: 3.20E-32',
 'bits: 0    eval: 0.00E+00',
 'bits: 0    qcov: 0.00',
 'bits: 0    qcov: 0.00',
 'bits: 82   eval: 1.60E-14',
 'bits: 75   eval: 3.90E-12',
 'bits: 120  qcov: 0.00',
 'bits: 104  qcov: 0.00',
 'bits: 185  eval: 2.30E-30',
 'bits: 176  eval: 3.40E-27',
 'bits: 0    qcov: 0.00',
 'bits: 0    qcov: 0.00',
 'bits: 86   eval: 2.10E-08',
 'bits: 0    eval: 0.00E+00',
 'bits: 558  qcov: 0.00',
 'bits: 281  qcov: 0.00',
 'bits: 105  eval: 1.40E-22',
 'bits: 81   eval: 4.30E-17',
 'bits: 144  qcov: 1.00',
 'bits: 75   qcov: 0.00',
 'bits: 186  eval: 1.00E-23',
 'bits: 180  eval: 1.40E-22',
 'bits: 297  qcov: 0.00',
 'bits: 134  qcov: 0.00',
 'bits: 186  eval: 1.00E-23',
 'bits: 180  eval: 1.40E-22',
 'bits: 161  qcov: 0.00',
 'bits: 0    qcov: 0.00',
 'bits: 97   eval: 4.90E-17',
 'bits: 93   eval: 4.10E-16',
 'bits: 161  qcov: 0.00',
 'bits: 130  qcov: 0.00',
 'bits: 186  eval: 3.60E-35',
 'bits: 0    eval: 0.00E+00',
 'bits: 0    qcov: 0.00',
 'bits: 0    qcov:

In [154]:
# df.query('query == "PC00001"')